In [ ]:
import torch
from source.ProcessData import ProcessData
from source.Model import Net
from torch import optim
from source.Utils import RandomSplit, Train, Validation,Predict, plot_loss, plot_predition
from torch_scatter import scatter_mean
from torch.optim.lr_scheduler import StepLR

In [ ]:
# get the processed data

dataset = ProcessData('data/')
data = dataset[0]

In [ ]:
# split the data to train, validate, test = 60:20:20
data = RandomSplit(data, 0.2, 0.2, 64)

In [ ]:
# loa the model
model = Net()

model

In [ ]:
total_trainable_params = sum(p.numel() for p in model.parameters())
print('total trainable params:', total_trainable_params)

In [ ]:
optimizer = optim.Adam(model.parameters(), lr=1.e-05)
scheduler = StepLR(optimizer, step_size=5, gamma=1.e-02)

In [ ]:
loss, epochs = {'train': [], 'valid': []}, []

# train the model and validate
for epoch in range(1, 201):
    epochs.append(epoch)
    loss['train'].append(Train(model, data['train'], optimizer, epoch))
    loss['valid'].append(Validation(model, data['valid'], epoch))
    scheduler.step()

In [ ]:
# plot the loss for train and validation
plot_loss(epochs, loss)

In [ ]:
# make predictions
predct_data = Predict(model, data['test'])

In [ ]:
# plot predictions
plot_predition(predct_data)